# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [130]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [131]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,torbay,47.6666,-52.7314,33.76,95,98,1.99,CA,1669161799
1,1,cherskiy,68.7500,161.3000,-38.49,100,42,1.07,RU,1669162034
2,2,naze,28.3667,129.4833,70.65,84,100,11.52,JP,1669162035
3,3,caravelas,-17.7125,-39.2481,71.64,89,69,5.03,BR,1669162036
4,4,olafsvik,64.8945,-23.7142,38.26,76,19,16.49,IS,1669162036


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [132]:
#
cities = city_data_df[["Lat", "Lng"]]
city_name = city_data_df["City"]
humidity = city_data_df["Humidity"]

# Configure the map plot
fig = gmaps.figure(center=(0, 0), zoom_level=2.20, layout={'width': '1200px', 'height': '500px'})

# Display the map
heat_layer = gmaps.heatmap_layer(cities, weights=humidity.values, dissipating=False, max_intensity=90, point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='1200px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [133]:
# Narrow down cities that fit criteria and drop any results with null values
dream_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70)
                                    & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)
                                    ]
dream_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,cabo san lucas,22.8909,-109.9124,79.52,66,0,6.91,MX,1669161767
72,72,salalah,17.0151,54.0924,75.29,57,0,3.44,OM,1669162081
99,99,lompoc,34.6391,-120.4579,71.78,50,0,8.05,US,1669162101
270,270,abu dhabi,24.4667,54.3667,75.31,68,0,2.30,AE,1669162219
544,544,miandrivazo,-19.5167,45.4667,74.48,71,0,1.97,MG,1669162416


### Step 3: Create a new DataFrame called `hotel_df`.

In [134]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = dream_cities_df.copy(deep=True)
#hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Lat"] = ""
hotel_df["Hotel Lng"] = ""
hotel_df["Rating"] = ""
hotel_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng,Rating
48,48,cabo san lucas,22.8909,-109.9124,79.52,66,0,6.91,MX,1669161767,,,,
72,72,salalah,17.0151,54.0924,75.29,57,0,3.44,OM,1669162081,,,,
99,99,lompoc,34.6391,-120.4579,71.78,50,0,8.05,US,1669162101,,,,
270,270,abu dhabi,24.4667,54.3667,75.31,68,0,2.30,AE,1669162219,,,,
544,544,miandrivazo,-19.5167,45.4667,74.48,71,0,1.97,MG,1669162416,,,,


### Step 4: For each city, use the Google Places API to find the first hotel located within 5,000 metres of your coordinates.

In [135]:
# Set parameters to search for a hotel
params = {
    "keyword": "hotel",
    "radius": 5000,
    "key": gkey
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Lat"] = name_address["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Hotel Lng"] = name_address["results"][0]["geometry"]["location"]["lng"]
        hotel_df.loc[index, "Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Solmar Resort
salalah - nearest hotel: Salalah Gardens Hotel
lompoc - nearest hotel: O'Cairns Inn & Suites
abu dhabi - nearest hotel: Sheraton Khalidiya Hotel
miandrivazo - nearest hotel: Hôtel Arc En Ciel Miandrivazo


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng,Rating
48,48,cabo san lucas,22.8909,-109.9124,79.52,66,0,6.91,MX,1669161767,Solmar Resort,22.876556,-109.903533,4.5
72,72,salalah,17.0151,54.0924,75.29,57,0,3.44,OM,1669162081,Salalah Gardens Hotel,17.02285,54.064747,4.8
99,99,lompoc,34.6391,-120.4579,71.78,50,0,8.05,US,1669162101,O'Cairns Inn & Suites,34.638258,-120.446835,4.5
270,270,abu dhabi,24.4667,54.3667,75.31,68,0,2.30,AE,1669162219,Sheraton Khalidiya Hotel,24.475108,54.349842,3.8
544,544,miandrivazo,-19.5167,45.4667,74.48,71,0,1.97,MG,1669162416,Hôtel Arc En Ciel Miandrivazo,-19.536976,45.45964,3.4


In [129]:
# Look at data
#name_address["results"][0]["geometry"]["location"]["lat"]
#name_address["results"][0]["rating"]
#name_address["results"][0]["name"]
#name_address["results"][0]["vicinity"]
#name_address

In [136]:
# Create list for Hotel Heatmap Hover Info
hotel_info = []

# Loop through Hotels to create the list
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get hotel info from the DataFrame
    hotel_name = row["Hotel Name"]
    city = row["City"]
    country = row["Country"]

    hotel_info.append({
                        "Hotel Info": hotel_name + ": " + city + ", " + country
    })

hotel_info

[{'Hotel Info': 'Solmar Resort: cabo san lucas, MX'},
 {'Hotel Info': 'Salalah Gardens Hotel: salalah, OM'},
 {'Hotel Info': "O'Cairns Inn & Suites: lompoc, US"},
 {'Hotel Info': 'Sheraton Khalidiya Hotel: abu dhabi, AE'},
 {'Hotel Info': 'Hôtel Arc En Ciel Miandrivazo: miandrivazo, MG'}]

In [137]:
# Create DataFrame for Hotel Info
hotel_info_df = pd.DataFrame(hotel_info)
hotel_info_df

,Hotel Info
0,"Solmar Resort: cabo san lucas, MX"
1,"Salalah Gardens Hotel: salalah, OM"
2,"O'Cairns Inn & Suites: lompoc, US"
3,"Sheraton Khalidiya Hotel: abu dhabi, AE"
4,"Hôtel Arc En Ciel Miandrivazo: miandrivazo, MG"


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [138]:
# Store latitude and longitude in hotel_locations
hotel_locations = hotel_df[["Hotel Lat","Hotel Lng"]]
#hotel_city = ["City"]
#hotel_country = ["Country"]
humidity = hotel_df["Humidity"]
hotel_names = hotel_df["Hotel Name"]
hotel_info = hotel_info_df["Hotel Info"]

# Configure the map plot
fig = gmaps.figure(center=(0, 0), zoom_level=2.20, layout={'width': '1200px', 'height': '500px'})

# Display the map
heat_layer = gmaps.heatmap_layer(hotel_locations, weights=humidity.values, 
                                dissipating=False, max_intensity=90,
                                point_radius=3)

#marker_layer = gmaps.marker_layer(hotel_locations, hover_text=hotel_names.values)
marker_layer = gmaps.marker_layer(hotel_locations, hover_text=hotel_info)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='500px', width='1200px'))